# Extract least privilege policy

In [70]:
with open('p.log','r') as f:
    linelist = f.readlines()
suffix = '(No such file or directory)\n'
res = [line for line in linelist if line.startswith('open') and not line.endswith(suffix)]
with open('p_after_extraction.log', 'w') as f:
    for line in res:
        f.write(line)

# Setfattr to each entry

In [71]:
dic = {}
skiplist = ["/dev", "/proc", "/lib", "/events", "/mnt", "/run", "/lvm", "/conf", "/usr", "/bin"]
for line in res:
    path = line.split('"')[1]
    shouldSkip = False
    for start in skiplist:
        if path.startswith(start):
            print('skip ' + start)
            shouldSkip = True
        if path == '/':
            print('skip /')
            shouldSkip = True
    if shouldSkip or path == '/etc/nshadow':
        continue
    middle = line.split('"')[2]
    permission = middle[2:].split(')')[0].split('|')
    if 'O_CLOEXEC' in permission:
        permission.remove('O_CLOEXEC')
    if 'O_NONBLOCK' in permission:
        permission.remove('O_NONBLOCK')
    if 'O_NOCTTY' in permission:
        permission.remove('O_NOCTTY')
    if 'O_DIRECTORY' in permission:
        permission.remove('O_DIRECTORY')
    dic[path] = permission[0]

skip /lib
skip /lib
skip /lib
skip /lib
skip /lib
skip /lib
skip /lib
skip /lib
skip /proc
skip /usr
skip /proc
skip /lib
skip /lib
skip /lib
skip /lib
skip /lib
skip /lib
skip /usr
skip /usr
skip /lib
skip /usr
skip /usr
skip /lib
skip /usr
skip /lib
skip /lib
skip /lib
skip /lib
skip /lib
skip /lib
skip /usr
skip /usr
skip /usr
skip /usr
skip /usr
skip /usr
skip /usr
skip /usr
skip /usr
skip /usr
skip /lib
skip /usr
skip /usr
skip /usr
skip /usr
skip /usr
skip /usr
skip /usr
skip /usr
skip /usr
skip /usr
skip /usr
skip /dev
skip /lib
skip /lib
skip /usr
skip /lib
skip /lib
skip /lib
skip /lib
skip /lib
skip /dev
skip /dev
skip /dev
skip /dev


In [72]:
for d in dic.keys():
    print(d)

/etc/ld.so.cache
/etc/nsswitch.conf
/etc/passwd
/var/lib/sss/mc/passwd
/etc/pam.d/passwd
/etc/pam.d/common-password
/etc/localtime
/etc/pam.d/other
/etc/pam.d/common-auth
/etc/pam.d/common-account
/etc/pam.d/common-session
/etc/security/pwquality.conf
/etc/krb5.conf
/etc/login.defs
/etc/shadow
/etc/.pwd.lock
/etc/nshadow


In [74]:
dic

{'/etc/ld.so.cache': 'O_RDONLY',
 '/etc/nsswitch.conf': 'O_RDONLY',
 '/etc/passwd': 'O_RDONLY',
 '/var/lib/sss/mc/passwd': 'O_RDONLY',
 '/etc/pam.d/passwd': 'O_RDONLY',
 '/etc/pam.d/common-password': 'O_RDONLY',
 '/etc/localtime': 'O_RDONLY',
 '/etc/pam.d/other': 'O_RDONLY',
 '/etc/pam.d/common-auth': 'O_RDONLY',
 '/etc/pam.d/common-account': 'O_RDONLY',
 '/etc/pam.d/common-session': 'O_RDONLY',
 '/etc/security/pwquality.conf': 'O_RDONLY',
 '/etc/krb5.conf': 'O_RDONLY',
 '/etc/login.defs': 'O_RDONLY',
 '/etc/shadow': 'O_RDONLY',
 '/etc/.pwd.lock': 'O_WRONLY',
 '/etc/nshadow': 'O_WRONLY'}

In [83]:
# Find directories
directory = set()
for path in dic.keys():
    sep = path.split("/")[1:]
    for i in range(len(sep)-1):
        dir_path = '/'+'/'.join(sep[:i+1])
        directory.add(dir_path)
        

In [84]:
directory

{'/etc',
 '/etc/pam.d',
 '/etc/security',
 '/var',
 '/var/lib',
 '/var/lib/sss',
 '/var/lib/sss/mc'}

In [85]:
with open('passwd.perm', 'w') as f:
    f.write('sudo setfattr -n security.mp4 -v target /usr/bin/passwd\n')
    f.write('sudo setfattr -n security.mp4 -v dir /home\n')
    f.write('sudo setfattr -n security.mp4 -v dir /home/lujin2\n')
    for di in directory:
        f.write('sudo setfattr -n security.mp4 -v dir ' + di + '\n')
    for path, permit in dic.items():
        if permit == "O_RDONLY":
            res = "sudo setfattr -n security.mp4 -v read-only "
        elif permit == "O_RDWR":
            res = "sudo setfattr -n security.mp4 -v read-write "
        elif permit == "O_WRONLY":
            res = "sudo setfattr -n security.mp4 -v write-only "
        else:
            print('ERROR! permit')
        f.write(res+path+'\n')

In [86]:
with open('passwd.perm.unload', 'w') as f:
    f.write('sudo setfattr -x security.mp4 /usr/bin/passwd\n')
    f.write('sudo setfattr -x security.mp4 /home\n')
    f.write('sudo setfattr -x security.mp4 /home/lujin2\n')
    for di in directory:
        f.write('sudo setfattr -x security.mp4 ' + di + '\n')
    for path in dic.keys():
        f.write('sudo setfattr -x security.mp4 '+path+'\n')